In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
import sys
#need attention
import multiprocessing.dummy as mp
from lxml import etree
import requests
import logging
import pickle
import json
import string
from zhon.hanzi import punctuation
from threading import Thread
from functools import wraps
import re
import warnings
import ddddocr
import random
import argparse
from  PIL import Image
import shutil
warnings.filterwarnings("ignore")

/Users/qianyuyang/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
xpath = {
    'freq':{
        'freq_select_button':'//div[@class="time-selection-controls"]/div[@class="selection-menu-button frequency"][1]',
        'freq_button':{
            'monthly':'//label[@for="monthly"]',
            'quarterly':'//label[@for="quarterly"]',
            'annual':'//label[@for="annual"]'
            }
    },
    'download':{
        'download_menu':'//div[@id="main_download_button"]',
        'download_csv_table':'//dl[@class="download_menu"]/dd/a[@id="csv_table"]'
    },
    'data':'//option[@class="link_option"]',
}

In [5]:
flag = 0
wait_time = 1
while flag < 3:
    flag = 0
    get_start_time = str(datetime.now())
    prefs = {"download.default_directory":"/Users/qianyuyang/Downloads/%s"%get_start_time}
    default_download = "/Users/qianyuyang/Downloads/%s"%get_start_time
    if not os.path.exists(default_download):
        os.mkdir(default_download)
    
    start_url = "https://www.eia.gov/beta/electricity/data/browser/"
    chromedriver = "/Volumes/TOSHIBA EXT/RA/energy_database/driver/chromedriver"
    options = webdriver.ChromeOptions()
    
    #prefs = {"download.default_directory":"/Users/qianyuyang/Downloads/%s"%1}
        

    # 谷歌文档提到需要加上这个属性来规避bug
    options.add_argument('--disable-gpu') 
    # 不加载图片, 提升速度
    #options.add_argument('blink-settings=imagesEnabled=false') 
    # 浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败
    #options.add_argument('--headless')
    #options.add_experimental_option("prefs", prefs)
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(executable_path = chromedriver,options = options)
    driver.get(start_url)
    #num_flag = True
    driver.maximize_window()
    for f in xpath['freq']['freq_button'].keys():
        print(f)
        driver.get(start_url)
        for file1 in os.listdir('../data/%s'%f):
            if len(os.listdir('../data/%s/%s'%(f,file1))) == 0:
                print(file1)
                os.rmdir('../data/%s/%s'%(f,file1))
        if not os.path.exists('../data/%s'%f):
            os.mkdir('../data/%s'%f)
        try:
            WebDriverWait(driver,20,1).until(EC.presence_of_element_located((By.XPATH,xpath['freq']['freq_select_button'])))
            time.sleep(wait_time)
            driver.find_element_by_xpath(xpath['freq']['freq_select_button']).click()
            time.sleep(wait_time)
            WebDriverWait(driver,10,1).until(EC.presence_of_element_located((By.XPATH,xpath['freq']['freq_button'][f])))
            driver.find_element_by_xpath(xpath['freq']['freq_button'][f]).click()
            time.sleep(wait_time)
        except:
            print(f"freq:{f}: error")
            continue
        
        html_source = driver.page_source
        label = etree.HTML(html_source)
        data_list = label.xpath(f"{xpath['data']}/text()")
        
        todolist = set(data_list) - set(os.listdir('../data/%s'%f))
        if len(todolist) == 0:
            flag += 1
            continue
        for j in todolist:
            
            idx = data_list.index(j)
            
            try:
                WebDriverWait(driver,10,1).until(EC.presence_of_all_elements_located((By.XPATH,xpath['data'])))[idx].click()
                time.sleep(wait_time)
            except:
                print(f"data:{j}: error")
                continue
            
            try:
                time.sleep(wait_time)
                WebDriverWait(driver,10,1).until(EC.presence_of_element_located((By.XPATH,xpath['download']['download_menu']))).click()
                time.sleep(wait_time)
                WebDriverWait(driver,10,1).until(EC.presence_of_element_located((By.XPATH,xpath['download']['download_csv_table']))).click()
                time.sleep(wait_time)
            except:
                print(f"download:{j}: error")
                continue
            if not os.path.exists('../data/%s/%s'%(f,j)):
                os.mkdir('../data/%s/%s'%(f,j))
            for file in os.listdir(f'{default_download}'):
                shutil.move(f'{default_download}/{file}','../data/%s/%s'%(f,j))
            time.sleep(wait_time)
        
    driver.quit()
    
    wait_time += 1
        
            
    # print(f)
    

monthly
quarterly
annual
download:Plant level data: error
monthly
quarterly
annual
monthly
quarterly
annual


In [6]:
driver.find_element_by_xpath(xpath['freq']['freq_select_button']).click()

In [8]:
driver.find_element_by_xpath(xpath['freq']['freq_button']['annual']).click()

In [21]:
driver.quit()

In [20]:
WebDriverWait(driver,10,1).until(EC.presence_of_all_elements_located((By.XPATH,xpath['data'])))[6].click()

In [17]:
EC.presence_of_all_elements_located((By.XPATH,xpath['data']))

<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [14]:
data_list.index("Net generation")

0

In [13]:
for i,j in enumerate(data_list):
    print(i,j)

0 Net generation
1 Total consumption
2 Total consumption (Btu)
3 Consumption for electricity generation
4 Consumption for electricity generation (Btu)
5 Consumption for useful thermal output
6 Consumption for useful thermal output (Btu)
7 Plant level data
8 Retail sales of electricity
9 Revenue from retail sales of electricity
10 Average retail price of electricity
11 Number of customer accounts
12 Fossil-fuel stocks for electricity generation
13 Receipts of fossil fuels by electricity plants
14 Receipts of fossil fuels by electricity plants (Btu)
15 Average cost of fossil fuels for electricity generation
16 Average cost of fossil fuels for electricity generation (per Btu)
17 Quality of fossil fuels in electricity generation : sulfur content
18 Quality of fossil fuels in electricity generation : ash content
19 1.1 Net generation by energy source: total - all sectors
20 1.1.a Net generation by renewable sources: total - all sectors
21 1.2.a Net generation by energy source: electric util

In [7]:
# select data frequency
driver.find_element_by_xpath('//div[@class="time-selection-controls"]/div[@class="selection-menu-button frequency"][1]').click()

In [8]:

driver.find_element_by_xpath('//label[@for="quarterly"]').click()

In [35]:
# download
driver.find_element_by_xpath('//div[@id="main_download_button"]').click()

In [36]:
# choose the attachment file format
driver.find_element_by_xpath('//dl[@class="download_menu"]/dd/a[@id="csv_table"]').click()

In [34]:
# choose pre-genrated report
driver.find_elements_by_xpath('//option[@class="link_option"]')[20].click()

In [ ]:
# choose dataset
driver.find_elements_by_xpath('//optgroup/option')[1].click()

In [20]:
# click the slect dataset
driver.find_elements_by_xpath('//optgroup[1]')
html_source = driver.page_source
label = etree.HTML(html_source)
all_dataset_name = label.xpath('//optgroup/option/text()')

In [21]:
all_dataset_name

['Net generation',
 'Total consumption',
 'Total consumption (Btu)',
 'Consumption for electricity generation',
 'Consumption for electricity generation (Btu)',
 'Consumption for useful thermal output',
 'Consumption for useful thermal output (Btu)',
 'Plant level data',
 'Retail sales of electricity',
 'Revenue from retail sales of electricity',
 'Average retail price of electricity',
 'Number of customer accounts',
 'Fossil-fuel stocks for electricity generation',
 'Receipts of fossil fuels by electricity plants',
 'Receipts of fossil fuels by electricity plants (Btu)',
 'Average cost of fossil fuels for electricity generation',
 'Average cost of fossil fuels for electricity generation (per Btu)',
 'Quality of fossil fuels in electricity generation : sulfur content',
 'Quality of fossil fuels in electricity generation : ash content']

In [ ]:
<div id="frequency-selection-button-container" class="selection-menu-button frequency open">
<div class="text">Monthly</div>
	</div>
//div[@class="selection-menu-button frequency open"]

In [ ]:
//div[@class="time-selection-controls"]/div[@class="selection-menu-button frequency open"]

In [ ]:
//label[@for="annual"]

In [ ]:
div[@id="main_download_button"]

In [ ]:
//dl[@class="download_menu"]/dd/a[@id="csv_table"]

In [ ]:
//optgroup

In [ ]:
<optgroup label="Electricity generation &amp; consumption (EIA-906/EIA-920/EIA-923)">
					<option class="link_option" value="0">Net generation</option>
					<option class="link_option" value="2">Total consumption</option>
					<option class="link_option" value="8">Total consumption (Btu)</option>
					<option class="link_option" value="3">Consumption for electricity generation</option>
					<option class="link_option" value="9">Consumption for electricity generation (Btu)</option>
					<option class="link_option" value="4">Consumption for useful thermal output</option>
					<option class="link_option" value="10">Consumption for useful thermal output (Btu)</option>
					<option class="link_option" value="1">Plant level data</option>
				</optgroup>